### 1. Dataset

In [22]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = "".join([page.extract_text() for page in reader.pages])
    return text

shona_text = extract_text_from_pdf("shona_article.pdf")

### 2. Vocabulary

In [23]:
from keras.preprocessing.text import Tokenizer

# Tokenize and preprocess
tokenizer = Tokenizer()
tokenizer.fit_on_texts([shona_text])
total_words = len(tokenizer.word_index) + 1


### 3. Word Embeddings

In [24]:
from gensim.models import Word2Vec

# Train word2vec model with gensim
sentences = [sentence.split() for sentence in shona_text.split('.')]
model_gensim = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
model_gensim.save("word2vec.model")


### 4. RNN Models

##### With its own embedding layer:

In [25]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model1 = Sequential()
model1.add(Embedding(total_words, 100, input_length=5))
model1.add(Bidirectional(LSTM(150, return_sequences=True)))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dense(total_words, activation='softmax'))

model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


##### With pre-trained embeddings:

In [26]:
# Load the word2vec model
import numpy as np
model_gensim = Word2Vec.load("word2vec.model")
embedding_matrix = np.zeros((total_words, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = model_gensim.wv[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    except KeyError:
        # Word not present in gensim model, a zero embedding will be used for this word
        pass


model2 = Sequential()
model2.add(Embedding(total_words, 100, weights=[embedding_matrix], input_length=5, trainable=False))
model2.add(Bidirectional(LSTM(150, return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(LSTM(100))
model2.add(Dense(total_words, activation='softmax'))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


### 5. Training & Testing

In [27]:
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

input_sequences = []
for line in shona_text.split('.'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

input_sequences = np.array(pad_sequences(input_sequences, maxlen=6, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Training Model 1
history1 = model1.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

# Training Model 2
history2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, verbose=1)

Epoch 1/100
3062/3062 [==============================] - 727s 231ms/step - loss: 3.5916 - accuracy: 0.5750 - val_loss: 3.4813 - val_accuracy: 0.5766
Epoch 2/100
3062/3062 [==============================] - 172s 56ms/step - loss: 3.1849 - accuracy: 0.5839 - val_loss: 3.5007 - val_accuracy: 0.5852
Epoch 3/100
3062/3062 [==============================] - 178s 58ms/step - loss: 2.9816 - accuracy: 0.5896 - val_loss: 3.6111 - val_accuracy: 0.5901
Epoch 4/100
3062/3062 [==============================] - 210s 69ms/step - loss: 2.7449 - accuracy: 0.6007 - val_loss: 3.6738 - val_accuracy: 0.5971
Epoch 5/100
3062/3062 [==============================] - 180s 59ms/step - loss: 2.4759 - accuracy: 0.6148 - val_loss: 3.8118 - val_accuracy: 0.6010
Epoch 6/100
3062/3062 [==============================] - 166s 54ms/step - loss: 2.2010 - accuracy: 0.6304 - val_loss: 3.9052 - val_accuracy: 0.6025
Epoch 7/100
3062/3062 [==============================] - 159s 52ms/step - loss: 1.9477 - accuracy: 0.6493 - val

3062/3062 [==============================] - 177s 58ms/step - loss: 0.1943 - accuracy: 0.9504 - val_loss: 5.4160 - val_accuracy: 0.6128
Epoch 57/100
3062/3062 [==============================] - 178s 58ms/step - loss: 0.1922 - accuracy: 0.9510 - val_loss: 5.4156 - val_accuracy: 0.6136
Epoch 58/100
3062/3062 [==============================] - 183s 60ms/step - loss: 0.1921 - accuracy: 0.9504 - val_loss: 5.4073 - val_accuracy: 0.6137
Epoch 59/100
3062/3062 [==============================] - 178s 58ms/step - loss: 0.1898 - accuracy: 0.9516 - val_loss: 5.4530 - val_accuracy: 0.6148
Epoch 60/100
3062/3062 [==============================] - 180s 59ms/step - loss: 0.1903 - accuracy: 0.9511 - val_loss: 5.4782 - val_accuracy: 0.6129
Epoch 61/100
3062/3062 [==============================] - 165s 54ms/step - loss: 0.1879 - accuracy: 0.9516 - val_loss: 5.4784 - val_accuracy: 0.6137
Epoch 62/100
3062/3062 [==============================] - 159s 52ms/step - loss: 0.1873 - accuracy: 0.9513 - val_loss: 

3062/3062 [==============================] - 110s 36ms/step - loss: 2.4678 - accuracy: 0.5892 - val_loss: 4.3839 - val_accuracy: 0.5847
Epoch 12/100
3062/3062 [==============================] - 109s 35ms/step - loss: 2.3987 - accuracy: 0.5911 - val_loss: 4.4551 - val_accuracy: 0.5859
Epoch 13/100
3062/3062 [==============================] - 109s 35ms/step - loss: 2.3361 - accuracy: 0.5947 - val_loss: 4.5479 - val_accuracy: 0.5872
Epoch 14/100
3062/3062 [==============================] - 110s 36ms/step - loss: 2.2792 - accuracy: 0.5977 - val_loss: 4.5979 - val_accuracy: 0.5857
Epoch 15/100
3062/3062 [==============================] - 110s 36ms/step - loss: 2.2286 - accuracy: 0.6013 - val_loss: 4.6256 - val_accuracy: 0.5841
Epoch 16/100
3062/3062 [==============================] - 109s 36ms/step - loss: 2.1796 - accuracy: 0.6049 - val_loss: 4.6660 - val_accuracy: 0.5865
Epoch 17/100
3062/3062 [==============================] - 107s 35ms/step - loss: 2.1364 - accuracy: 0.6101 - val_loss: 

3062/3062 [==============================] - 107s 35ms/step - loss: 1.2709 - accuracy: 0.7346 - val_loss: 5.9027 - val_accuracy: 0.5885
Epoch 67/100
3062/3062 [==============================] - 108s 35ms/step - loss: 1.2670 - accuracy: 0.7345 - val_loss: 5.9255 - val_accuracy: 0.5856
Epoch 68/100
3062/3062 [==============================] - 107s 35ms/step - loss: 1.2579 - accuracy: 0.7357 - val_loss: 5.9161 - val_accuracy: 0.5878
Epoch 69/100
3062/3062 [==============================] - 107s 35ms/step - loss: 1.2523 - accuracy: 0.7363 - val_loss: 5.9299 - val_accuracy: 0.5876
Epoch 70/100
3062/3062 [==============================] - 107s 35ms/step - loss: 1.2448 - accuracy: 0.7376 - val_loss: 5.9644 - val_accuracy: 0.5873
Epoch 71/100
3062/3062 [==============================] - 108s 35ms/step - loss: 1.2395 - accuracy: 0.7383 - val_loss: 5.9661 - val_accuracy: 0.5879
Epoch 72/100
3062/3062 [==============================] - 109s 36ms/step - loss: 1.2294 - accuracy: 0.7411 - val_loss: 

### Model Evaluation

In [28]:
val_loss_model1 = history1.history['val_loss'][-1]
val_loss_model2 = history2.history['val_loss'][-1]

print(f"Validation Loss for Model 1: {val_loss_model1}")
print(f"Validation Loss for Model 2: {val_loss_model2}")


Validation Loss for Model 1: 5.767948150634766
Validation Loss for Model 2: 6.362008571624756


In [29]:
if val_loss_model1 < val_loss_model2:
    best_model = model1
    best_model_name = "best_model1.h5"
else:
    best_model = model2
    best_model_name = "best_model2.h5"

best_model.save(best_model_name)
print(f"Saved the best model as {best_model_name}")


C:\Users\tadiw\anaconda3\envs\deeplearning\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Saved the best model as best_model1.h5


In [30]:
params_model1 = model1.count_params()
params_model2 = model2.count_params()

print(f"Model 1 has {params_model1} parameters.")
print(f"Model 2 has {params_model2} parameters.")


Model 1 has 3277610 parameters.
Model 2 has 3277610 parameters.


In [33]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# Load the previously saved model
model = load_model('best_model2.h5')

def predict_next_words(model, tokenizer, text, num_words=1):
    """
    Predict the next set of words using the trained model.
    
    Args:
    - model (keras.Model): The trained model.
    - tokenizer (Tokenizer): The tokenizer object used for preprocessing.
    - text (str): The input text.
    - num_words (int): The number of words to predict.

    Returns:
    - str: The predicted words.
    """
    for _ in range(num_words):
        # Tokenize and pad the text
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = pad_sequences([sequence], maxlen=5, padding='pre')
        
        # Predict the next word
        predicted_probs = model.predict(sequence, verbose=0)
        predicted = np.argmax(predicted_probs, axis=-1)
        
        # Convert the predicted word index to a word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        
        # Append the predicted word to the text
        text += " " + output_word

    return ' '.join(text.split(' ')[-num_words:])


# Prompt the user for input
user_input = input("Please type five words in Shona: ")

# Predict the next words
predicted_words = predict_next_words(model, tokenizer, user_input, num_words=3)
print(f"The next words might be: {predicted_words}")


Please type five words in Shona: Makadini zvenyu vanofara here vamwe
The next words might be: kataura kuti urambe
